In [ ]:
import os
import sys
import pandas as pd
pd.options.mode.copy_on_write = True 

from pathlib import Path
import cdata_utils
import numpy as np
import matplotlib.pyplot as plt
import cdata_utils.utils
import datetime


import json


import lifelines
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

from lifelines import KaplanMeierFitter


import sksurv
#from sksurv.preprocessing import OneHotEncoder
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.preprocessing import OneHotEncoder, encode_categorical
from sksurv.util import Surv

import sklearn
from sklearn.preprocessing import StandardScaler



#import cdata_utils.preprocess.read_and_clean_tabular
from cdata_utils.project_specific.psvd import (
    read_and_clean_PSVD_data__BL_consensus,
    read_and_clean_PSVD_data__BL_consensus_NEW,
    categorize_PSVD_data,
    exclude_patients,
    reorder_some_categorical_values, 
    table1_psvd, 
    table1_psvd_spleen, 
    descriptive_df_from_masks, 
    masks_for_endpoint_1__decompensation, 
    masks_for_endpoint_2__death,
    make_y_delta,
    drop_non_numeric_columns,
    table_of_valid_entries, 
    univariate_cox_ph_summary, 
    normalize_df,
    load_EP1_EP2_data,
    relevant_column_names,
    relevant_column_names_clinical,
    categorize_PSVD_clinical_data,
    load_clinical_data,
)

from cdata_utils.descriptive.basic_stats import (
    describe
)

from cdata_utils.descriptive.lifelines import extract_summary_data_from_lifelines


import cdata_utils.preprocess
import cdata_utils.project_specific
import cdata_utils.project_specific.psvd
from cdata_utils.utils import integerized

import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from matplotlib.backends.backend_pdf import PdfPages
 
# path info: 
if "cwatzenboeck" in os.getcwd(): # desktop 
    data_path = Path("/home/cwatzenboeck/Dropbox/work/data/livermodel/PSVD/")
    data_path_output=Path("/home/cwatzenboeck/data/psvd/output_coxph/")
else: # laptop 
    data_path = Path("/home/clemens/Dropbox/work/data/livermodel/PSVD/")
    # data_path = Path("/home/clemens/projects/project_liver_model/data/PSVD")


In [ ]:
# load data:
df1, df2 = load_EP1_EP2_data(data_path,  file_name = "data_PSVD_unified_3.xlsx")

df_c = load_clinical_data(data_path, file_name="data_PSVD_unified_3.xlsx", drop_modfied_colums=True)
df1c = df1.join(df_c.drop(columns=['Sex (1=male, 2=female)']))
# df2c = df2.join(df_c.drop(columns=['Sex (1=male, 2=female)']))

# filter data to relevant columns: EP 1
df1c_ = df1c.filter(regex="BL Location consensus binary cat.|^BL SPSS|BL segment IV|Age|LSM \(|PSVD cause 1|PSVD cause 2|status|event")  # cat BL segment IV 
df1c_["BL segment IV MW_0"] = ~df1c_["BL segment IV MW"] == 0; 
assert ~df1c_["BL segment IV MW"].isnull().any()
df1c_ = df1c_.drop(columns=["BL segment IV MW"])
print(df1c_.shape)
df1c_ = df1c_.dropna()
print(df1c_.shape)


# # filter data to relevant columns: EP 2
# df2c_ = df2c.filter(regex="Ascites|Sarcopenia|^BL SPSS|Age|LSM \(|PSVD cause 1|PSVD cause 2|status|event")  # cat BL segment IV 
# print(df2c_.shape)
# df2c_ = df2c_.dropna()
# print(df2c_.shape)


In [ ]:
# make model EP1:

cols = [  # 'BL SPSS consensus',
    'BL Location consensus binary cat.',
    'BL_Age',
    'LSM (kPa)',
    # 'PSVD cause 1 (CVID/autoimmune/inflammatory)',
    'PSVD cause 2 (drug induced/toxic)',
    'BL segment IV MW_0',
    'status',
    'event'
]

df1c_n = normalize_df(df1c_, columns_to_ignore=['BL SPSS consensus',
                                                'BL Location consensus binary cat.',
                                                # 'BL_Age',
                                                # 'LSM (kPa)',
                                                'PSVD cause 1 (CVID/autoimmune/inflammatory)',
                                                'PSVD cause 2 (drug induced/toxic)',
                                                'BL segment IV MW_0',
                                                'status',
                                                'event'
                                                ])


df = df1c_n.copy()
df = df[cols]

cph = lifelines.CoxPHFitter()
df["event"] = df["event"]*12
cph.fit(df, duration_col="event", event_col="status")
cph.print_summary()
results_df1_A, results_df1_B = extract_summary_data_from_lifelines(cph)

####df.to_csv(data_path_output / "cox_ph_EP1_data.csv")
results_df1_A.transpose().to_excel(data_path_output / "cox_ph_EP1_multiCoxPH_clinical_and_radio_fit_summary.xlsx")
results_df1_B.to_excel(data_path_output / "cox_ph_EP1_multiCoxPH_clinical_and_radio_parameters_summary.xlsx")

cph.plot()
# Create a larger figure
fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the figsize as needed
cph.plot(ax=ax)
plt.tight_layout()
plt.savefig(data_path_output /  "cox_ph_EP1_multiCoxPH_clinical_and_radio_HR.pdf")

In [ ]:
results_df1_B.sort_values("p")

In [ ]:


# Assuming 'cph' is your fitted CoxPHFitter model object

df = df1c_.copy()
df = df[cols]
cph = lifelines.CoxPHFitter()
df["event"] = df["event"]*12
cph.fit(df, duration_col="event", event_col="status")


x_max = 12 * 10  # x_max = 60
y_min = 0.25
y_max = 1

ylab = r"est. probability of no decompensation $\hat{S}(t)$"
xlab = r"time $t$ [months]"

# Create a 2x2 subplot grid
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Helper function to set axis properties
def set_axis_properties(ax, x_max, y_min, y_max, set_xlabel=True, set_ylabel=True, rm_xticks_labels = False, rm_yticks_labels = False):
    ax.set_xlim(0, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(np.arange(0, x_max+1, 12))
    ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
    if set_xlabel:
        ax.set_xlabel(xlab)
    if set_ylabel:
        ax.set_ylabel(ylab)
    if rm_xticks_labels:
        labels = [item.get_text() for item in ax.get_xticklabels()]
        empty_string_labels = ['']*len(labels)
        ax.set_xticklabels(empty_string_labels)
    if rm_yticks_labels:
        labels = [item.get_text() for item in ax.get_yticklabels()]
        empty_string_labels = ['']*len(labels)
        ax.set_yticklabels(empty_string_labels)
        

# Plot 00
mean, std, q25, q50, q75 = df.describe()["BL_Age"].transpose()[["mean", "std", "25%", "50%", "75%"]].to_numpy()
plot00 = cph.plot_partial_effects_on_outcome(
    covariates="BL_Age", 
    values=[q25, q50, q75], 
    #values=[mean - 1.0*std, mean, mean + 1.0*std], 
    cmap='coolwarm',
    ax=axes[0, 0])
# axes[0, 0].set_title("age")
# axes[0, 0].legend(labels=[f"mean - std = {(mean - std):4.2f}", "mean", "mean + std"], loc='lower left')
axes[0, 0].legend(labels=[f"age = {(q25):2.0f} years (25% quantile)",
                          f"age = {(q50):2.0f} years (median)",
                          f"age = {(q75):2.0f} years (75% quantile)"], loc='lower left')
set_axis_properties(axes[0, 0], x_max, y_min, y_max, set_xlabel=False, set_ylabel=True, rm_xticks_labels = True)


# # Plot 01

plot00 = cph.plot_partial_effects_on_outcome(
    covariates=[#'PSVD cause 1 (CVID/autoimmune/inflammatory)',
                'PSVD cause 2 (drug induced/toxic)'], 
    values=np.array([[0],
                     [1]]),
    cmap='coolwarm',
    ax=axes[0, 1])
# axes[0, 1].set_title("PSVD cause")
# axes[0, 1].legend(labels=[
#      'PSVD cause 1 (CVID/autoimmune/inflammatory)',
#      'neither PSVD cause 1 nor PSVD cause 2',
#      'PSVD cause 2 (drug induced/toxic)'
#                            ], loc='lower left')
set_axis_properties(axes[0, 1], x_max, y_min, y_max, set_xlabel=False, set_ylabel=True, rm_xticks_labels = True, rm_yticks_labels = True)



# # Plot 10
mean, std, q25, q50, q75 = df.describe()["LSM (kPa)"].transpose()[["mean", "std", "25%", "50%", "75%"]].to_numpy()
plot00 = cph.plot_partial_effects_on_outcome(
    covariates="LSM (kPa)", 
    values=[q25, q50, q75], 
    #values=[mean - 1.0*std, mean, mean + 1.0*std], 
    cmap='coolwarm',
    ax=axes[1, 0])
# axes[1, 0].set_title("LSM")
# axes[0, 0].legend(labels=[f"mean - std = {(mean - std):4.2f}", "mean", "mean + std"], loc='lower left')
axes[1, 0].legend(labels=[f"LSM = {(q25):4.1f} kPa  (25% quantile)",
                          f"LSM = {(q50):4.1f} kPa  (median)",
                          f"LSM = {(q75):4.1f} kPa  (75% quantile)"], loc='lower left')
set_axis_properties(axes[1, 0], x_max, y_min, y_max, set_xlabel=True, set_ylabel=True)


# # Plot 11
plot00 = cph.plot_partial_effects_on_outcome(
    covariates=['BL segment IV MW_0', 'BL Location consensus binary cat.'], 
    values=np.array([#[0, 0],
                     [1, 0],
                     [0, 1],
                     #[1, 1]
                     ]),
    cmap='coolwarm',
    ax=axes[1, 1])
# axes[1, 1].set_title("SPSS/thrombosis")
axes[1, 1].legend(labels=["segment IV = 'atrophy' or 'hypertrophy'", 
                          r"thrombosis = any('SMV','SV','extrahep.','intrahep.','LPV','RPV')", 
                          ], loc='lower left')
set_axis_properties(axes[1, 1], x_max, y_min, y_max, set_xlabel=True, set_ylabel=True, rm_yticks_labels = True)


# Adjust layout
plt.tight_layout()

# Save the plot to a PDF file
save_figure = True
if save_figure:
    with PdfPages(data_path_output /  "cox_ph_EP1_multiCoxPH_clinical_and_radio_model_plot.pdf") as pdf:
        pdf.savefig(fig)

# Show the plot (optional)
plt.show()

# Close the plot
plt.close(fig)